<a href="https://www.kaggle.com/code/eavprog/tr-order-lama-simple?scriptVersionId=94726295" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Загрузка библиотек

In [15]:
!pip install -U lightautoml

In [16]:
!pip install yfinance

In [17]:
pip install git+https://github.com/prog815/progOrder

  Cloning https://github.com/prog815/progOrder to /tmp/pip-req-build-3nkks7w6
  Running command git clone --filter=blob:none --quiet https://github.com/prog815/progOrder /tmp/pip-req-build-3nkks7w6
  Resolved https://github.com/prog815/progOrder to commit da7c7d38cd9609ee97bcf13032c088475c8a259b
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [18]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import progOrder

# Загрузка котировок

In [19]:
data = yf.download('SBER.ME',period='10y')
data

[*********************100%***********************]  1 of 1 completed


ModuleNotFoundError: No module named 'pandas.io.formats.string'

ImportError: cannot import name 'TableFormatter' from 'pandas.io.formats.format' (/opt/conda/lib/python3.7/site-packages/pandas/io/formats/format.py)

# Подготовка данных для модели

In [20]:
?progOrder.getBuySellForData

Signature:
progOrder.getBuySellForData(
    ohlc,
    orderLifeLen=50,
    dSL=0.001,
    dTP=0.0005,
    SP=0.07,
)
Docstring:
Расчет ордеров на покупку и продажу

Аргументы:
    ohlc - матрица Open,High,Low,Close
    orderLifeLen - время жизни ордера
    dSL - доля на stop-loss (относительно close[-1])
    dTP - доля на take-profit (относительно close[-1])
    SP - своп абсолютно

Вовращает:
    buy,sell - два вектора (покупка и продажа):
        1 - удачный ордер
        0 - неудачный ордер

Замечание:
    Ордер ставится по закрытию close(n) после окончания точки (n).
    Если поставил ордер после окончания точки (n) то результат отработки будет buy(n) и sell(n).
    Смещать ряды не нужно.

Пример:
    b,s = getBuySellForData(ohlc,dSL=0.01)
File:      /opt/conda/lib/python3.7/site-packages/progOrder/__init__.py
Type:      function


In [21]:
window = 100 
test_part = 0.2

# подготовка входов
df = pd.DataFrame(index=data.index)
df['dClose'] = data['Close'].pct_change()

# оконный сдвиг
for name in df.columns:
    for n in range(1,window):
        df[name + "_" + str(n)] = df[name].shift(n)

# добавляем дату
df['dow'] = df.index.day_of_week
df['dow_sin'] = np.sin(2 * np.pi * df['dow'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['dow'] / 7)

df['dom'] = data.reset_index()['Date'].dt.day.values
df['dom_sin'] = np.sin(2 * np.pi * df['dom'] / 31)
df['dom_cos'] = np.cos(2 * np.pi * df['dom'] / 31)

df['month'] = data.reset_index()['Date'].dt.month.values
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

df['week'] = data.reset_index()['Date'].dt.isocalendar().week.values - 1
df['week_sin'] = np.sin(2 * np.pi * df['month'] / 53)
df['week_cos'] = np.cos(2 * np.pi * df['month'] / 53)

# размножаем на два типа ордеров
data_buy = df.copy()
data_sell = df.copy()

# добавляем таргет
ohlc = data.values[:,:4]
buy,sell = progOrder.getBuySellForData(ohlc,SP=0.05)
data_buy['Target'] = buy.astype(int)
data_sell['Target'] = sell.astype(int)

# выделяем размер тестовой

test_len = int((data.shape[0]-window-1)*test_part)

# выделяем обучающую

data_buy_train = data_buy.iloc[window+1:-test_len-1,:]
data_sell_train = data_sell.iloc[window+1:-test_len-1,:]

# выделяем тестовоую

data_buy_test = data_buy.iloc[-test_len-1:-1,:]
data_sell_test = data_sell.iloc[-test_len-1:-1,:]

# выделяем расчетную

data_buy_calc = data_buy.iloc[[-1],:]
data_sell_calc = data_sell.iloc[[-1],:]

data_buy_test

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=

ModuleNotFoundError: No module named 'pandas.io.formats.string'

ImportError: cannot import name 'TableFormatter' from 'pandas.io.formats.format' (/opt/conda/lib/python3.7/site-packages/pandas/io/formats/format.py)

In [22]:
# контроль средних
pd.DataFrame([data_buy_train.describe().loc['mean',:],data_buy_test.describe().loc['mean',:]],index=['train','test'])

ModuleNotFoundError: No module named 'pandas.io.formats.string'

ImportError: cannot import name 'TableFormatter' from 'pandas.io.formats.format' (/opt/conda/lib/python3.7/site-packages/pandas/io/formats/format.py)

In [23]:
# контроль стандартного отклонения
pd.DataFrame([data_buy_train.describe().loc['std',:],data_buy_test.describe().loc['std',:]],index=['train','test'])

ModuleNotFoundError: No module named 'pandas.io.formats.string'

ImportError: cannot import name 'TableFormatter' from 'pandas.io.formats.format' (/opt/conda/lib/python3.7/site-packages/pandas/io/formats/format.py)

# Создание модели

In [24]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from lightautoml.validation.np_iterators import TimeSeriesIterator

In [25]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve

def acc_score(y_true, y_pred, **kwargs):
    return accuracy_score(y_true, (y_pred > 0.5).astype(int), **kwargs)

def f1_metric(y_true, y_pred, **kwargs):
    return f1_score(y_true, (y_pred > 0.5).astype(int), **kwargs)

In [26]:
task = Task('binary')
roles = { 'target': 'Target'}
automl = TabularAutoML(task)

# Обучение модели

In [27]:
data_buy_train.index.values.astype(np.datetime64)

array(['2012-09-27T00:00:00.000000000', '2012-09-28T00:00:00.000000000',
       '2012-10-01T00:00:00.000000000', ...,
       '2020-05-13T00:00:00.000000000', '2020-05-14T00:00:00.000000000',
       '2020-05-15T00:00:00.000000000'], dtype='datetime64[ns]')

In [28]:
cv_iter = TimeSeriesIterator(data_buy_train.index.values.astype(np.datetime64), n_splits=7, sorted_kfold=False)
oof_pred = automl.fit_predict(data_buy_train,roles=roles,verbose=1,cv_iter=cv_iter)

[08:12:53] Stdout logging level is INFO.
[08:12:53] Task: binary

[08:12:53] Start automl preset with listed constraints:
[08:12:53] - time: 3600.00 seconds
[08:12:53] - CPU: 4 cores
[08:12:53] - memory: 16 GB

[08:12:53] Train data shape: (1911, 113)

[08:12:54] Layer 1 train process start. Time left 3599.07 secs
[08:12:54] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...


AssertionError: Pipeline finished with 0 models for some reason.
Probably one or more models failed

In [ ]:
oof_pred.data.max()

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(x=data_buy_train['Target'].values,y=oof_pred.data[:,0])

In [ ]:
(oof_pred.data > 0.02).mean()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
roc_auc_score(data_buy_train['Target'].values,oof_pred.data[:,0])

In [ ]:
fpr,tpr,thresholds = roc_curve(data_buy_train['Target'].values,oof_pred.data[:,0])
plt.plot(fpr,tpr)
plt.plot([0,1],[0,1],':')

In [ ]:
import optuna

In [ ]:
def f1_metric_trh(y_true, y_pred, threshold = 0.5, **kwargs):
    return f1_score(y_true, (y_pred > threshold).astype(int), **kwargs)

def objective(trial):
    threshold = trial.suggest_float("threshold", 0.01, 0.99)
    return f1_metric_trh(data_buy_train['Target'].values,oof_pred.data[:,0], threshold)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)


In [ ]:
study.best_params,f1_metric_trh(data_buy_train['Target'].values,oof_pred.data[:,0], study.best_params['threshold'])

# Тестирование модели

In [ ]:
pred_test = automl.predict(data_buy_test)

In [ ]:
f1_metric_trh(data_buy_test['Target'].values,pred_test.data[:,0], study.best_params['threshold'])